In [ ]:
# wide and embedding, 
# wide co-occurrence
# wide co-occurrence & interaction 


In [1]:
import gzip
import os
import tempfile

import numpy as np
import pandas as pd
# from six.moves import urllib
# from six.moves import xrange  # pylint: disable=redefined-builtin


import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist
from tensorflow.contrib import rnn


import math
import random
import time
import datetime

from keras.utils import np_utils

from sklearn import preprocessing



Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [2]:
# continuous and categorical feature split

# features on columns [0:split_idx] are continuous
# the rest are categorical
def conti_cate_split(dta_df, split_idx):
    
    col_cnt= dta_df.shape[1]
    conti_cols= range(split_idx)
    dis_cols=range(split_idx, col_cnt)
    
    return dta_df[conti_cols], dta_df[dis_cols]

def conti_normalization_train_dta(dta_df):
    
    return preprocessing.scale(dta_df)

def conti_normalization_test_dta(dta_df, train_df):
    
    mean_dim = np.mean(train_df, axis=0)
    std_dim = np.std(train_df, axis=0)
        
    df=pd.DataFrame()
    cols = train_df.columns
    idx=0
    
    for i in cols:
        df[i] = (dta_df[i]- mean_dim[idx])*1.0/std_dim[idx]
        idx=idx+1
        
    return df

files_list=["../dataset/dataset_demo/xtrain.csv", \
            "../dataset/dataset_demo/xtest.csv",\
            "../dataset/dataset_demo/ytrain.csv", \
            "../dataset/dataset_demo/ytest.csv"]

xtrain_df=pd.read_csv( files_list[0] ,sep=',', header=None)
xtest_df=pd.read_csv( files_list[1] ,sep=',', header=None)
ytrain_df=pd.read_csv( files_list[2] ,sep=',', header=None)
ytest_df=pd.read_csv( files_list[3] ,sep=',', header=None)

print xtrain_df.shape, xtest_df.shape, ytrain_df.shape, ytest_df.shape


# get training and testing data prepared

cxtrain, dxtrain = conti_cate_split(xtrain_df, 106)
cxtest, dxtest = conti_cate_split(xtest_df, 106)

cxtest = conti_normalization_test_dta(cxtest, cxtrain)
cxtrain = conti_normalization_train_dta(cxtrain)

print cxtrain.shape, dxtrain.shape
print cxtest.shape, dxtest.shape

# cxtrain = cxtrain.as_matrix() 
dxtrain = dxtrain.as_matrix().astype(int)
cxtest = cxtest.as_matrix()
dxtest = dxtest.as_matrix().astype(int)

ytest = np_utils.to_categorical( ytest_df.as_matrix() )
ytrain =np_utils.to_categorical( ytrain_df.as_matrix() )

(6938, 186) (771, 186) (6938, 1) (771, 1)
(6938, 106) (6938, 80)
(771, 106) (771, 80)


In [4]:
# wide, embedding and co-occurrence of features

class wide_embed_coocc_NN():
    
    def __init__(self, n_conti, n_disc, n_class,\
                 n_disc_voca, session,n_embedding, n_hidden_list, lr, l2):
        
        self.LEARNING_RATE = lr
                
        self.N_CLASS = n_class
        self.N_CONTI = n_conti
        self.N_DISC = n_disc
        self.N_DISC_VOCA = n_disc_voca
        self.N_EMBED = n_embedding
        self.L2 = l2
        
        self.N_HIDDEN_LAYERS = len(n_hidden_list)
        self.N_TOTAL = self.N_CONTI+ self.N_DISC*self.N_EMBED

        self.sess=session
        
        self.dx = tf.placeholder(tf.int32, [None, self.N_DISC])
        self.cx = tf.placeholder(tf.float32, [None, self.N_CONTI])
        self.y = tf.placeholder(tf.float32, [None, self.N_CLASS])
        self.lr = tf.placeholder(tf.float32, shape=())
        self.keep_prob = tf.placeholder(tf.float32)
        
#       wide part on categorical features
        self.dx_trans = tf.transpose( self.dx, [1,0] )
        
        for i in range(self.N_DISC):
            
            with tf.variable_scope("wide"+str(i)):
                w= tf.Variable(\
                tf.random_uniform([self.N_DISC_VOCA[i], self.N_CLASS],-1.0, 1.0) )
#                         tf.random_normal([self.N_DISC_VOCA, self.N_CLASS],\
#                         stddev=math.sqrt(2.0/float(self.N_DISC_VOCA))) )
                if i==0:
                    dx_wsum = tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
#                   L2
                    self.regularizer_wide = tf.nn.l2_loss(w)
                    
                else:
                    dx_wsum = dx_wsum + tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
#                   L2  
                    self.regularizer_wide = self.regularizer_wide + tf.nn.l2_loss(w)
                    
            
#       embedding of categorical features        
        tmp_embeded = []                
        for i in range(self.N_DISC):
            
            with tf.variable_scope("embedding"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA[i], self.N_EMBED],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )
                              
                tmp_embeded.append( tf.nn.embedding_lookup( w, self.dx_trans[i] ) )
                
        dx_embeded = tf.concat(tmp_embeded, 1)
        x_concate =  tf.concat( [self.cx,dx_embeded], 1 )
        
#       co-occurrence of categorical features
        for i in range(10):
            
            tmpvoca1 = self.N_DISC_VOCA[i] 
            
            for j in range(i+1,10):
                
                tmpvoca2 = self.N_DISC_VOCA[j]
                
                with tf.variable_scope("cooccur"+str(i)+str(j)):
                    
                    
#                   vector approximate of co-occurrence matrix 
                    w1 = tf.Variable(tf.random_normal([self.N_CLASS, tmpvoca1, 1],\
                                            stddev=math.sqrt(2.0/float( tmpvoca1 ))) )
                    
                    w2 = tf.Variable(tf.random_normal([self.N_CLASS, 1, tmpvoca2],\
                                            stddev=math.sqrt(2.0/float( tmpvoca2 ))) )
                    
                    cooc_mat = []
                    for k in range(self.N_CLASS):
                        cooc_mat.append( tf.reshape( tf.matmul(w1[k],w2[k]), [-1,1] ) )
                        
#                   n_class by voca1*voca2  
                    cooc_vec = tf.stack(cooc_mat)
                    cooc_vec = tf.reshape(cooc_vec, [self.N_CLASS, -1] )
                    cooc_vec = tf.transpose(cooc_vec,[1,0])
                    
#                   build the idx of co-occurrence features
                    cooc_idx = self.dx_trans[i] * tmpvoca2 + self.dx_trans[j]
    
                    if i==0 and j==1:
                        cooc_wsum  = tf.nn.embedding_lookup( cooc_vec, cooc_idx )
                        self.regularizer_cooc   = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
                    else:
                        cooc_wsum += tf.nn.embedding_lookup( cooc_vec, cooc_idx )
                        self.regularizer_cooc += (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
        
                    
# #     hidden layers
        with tf.variable_scope("h0"):
            w= tf.Variable(tf.random_normal([self.N_TOTAL, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(self.N_CONTI)))) 
            b= tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            h = tf.nn.relu( tf.add( tf.matmul(x_concate, w),b) )
        
                
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("layer"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                b= tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                h = tf.nn.relu( tf.add( tf.matmul(h, w),b) )
                
                
#       Regularization

#       dropout
#       h = tf.nn.dropout(h, self.keep_prob)

#       L2
        self.regularizer = tf.nn.l2_loss(w)
        
#       output layer  
        with tf.variable_scope("output"):
            
            w= tf.Variable(tf.random_normal([n_hidden_list[self.N_HIDDEN_LAYERS-1],\
                                             self.N_CLASS],\
            stddev=math.sqrt(2.0/float(n_hidden_list[self.N_HIDDEN_LAYERS-1])))) 
            b= tf.Variable(tf.zeros( [ self.N_CLASS ] ))
            
            h = tf.add( tf.matmul(h, w), dx_wsum ) #wide
            h = tf.add( h, cooc_wsum ) #co-occurrence
            h = tf.add( h, b )


            self.logit = h
     
    
#           L2  
            self.regularizer = self.regularizer + tf.nn.l2_loss(w) 
    
#       overall L2
        self.regularizer=self.regularizer + self.regularizer_wide + self.regularizer_cooc
         
    
#   initialize loss and optimization operations for training
    def train_ini(self):
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                                    logits=self.logit, labels=self.y) \
                                  + self.L2*self.regularizer )
        self.optimizer = \
        tf.train.AdadeltaOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#       tf.train.RMSPropOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdadeltaOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.GradientDescentOptimizer(learning_rate = self.lr).minimize(self.cost)
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
#   training on batch of data
    def train_batch(self, dx_batch,cx_batch, y_batch, keep_prob, lr):
        
        _,c = sess.run([self.optimizer,self.cost],\
                       feed_dict={self.dx:dx_batch, \
                                  self.cx:cx_batch, self.y:y_batch,\
                                  self.lr:lr,\
                                  self.keep_prob:keep_prob\
                                 })
        
        return c
    
#   initialize inference         
    def inference_ini(self):
        self.correct_prediction = tf.equal(tf.argmax(self.logit,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, "float"))
        
#   infer givn testing data    
    def inference(self, dx_test, cx_test, y_test, keep_prob):
        return sess.run([self.accuracy], feed_dict={self.dx:dx_test,\
                                                  self.cx:cx_test, self.y:y_test,\
                                                  self.keep_prob:keep_prob\
                                                   })
        
#   unit_test
    def test(self, dx_test, cx_test, y_test ):
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        return sess.run( [self.tmpshape], \
                 feed_dict={self.dx:dx_test, self.cx:cx_test, self.y:y_test})


In [ ]:
para_n_epoch = 1200

# tunable parameters
#   representation ability
para_n_hidden_list = [ 128,64,8 ]
#     128,16,8 ]
para_lr = 0.055
para_n_embedding = 2
#   regularization
para_batch_size = 32
para_keep_prob = 1.0
para_l2 = 0.15

# fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = [8]*para_n_disc

# evaluation parameters
para_eval_byepoch = 10


with tf.Session() as sess:
    
    wide_mlp_clf = wide_embed_coocc_NN( para_n_conti, para_n_disc, para_n_class, \
                      para_n_disc_voca, sess, para_n_embedding, para_n_hidden_list,\
                        para_lr, para_l2 )
    wide_mlp_clf.train_ini()
    wide_mlp_clf.inference_ini()
    
    para_cur_lr=para_lr
    
    total_cnt= np.shape(cxtrain)[0]
    total_batch = int(total_cnt/para_batch_size)

    total_idx=range(total_cnt)

    
    # Training cycle
    for epoch in range(para_n_epoch):
        
        tmpc=0.0
        
#       shuffle traning dataset each epoch  
        np.random.shuffle(total_idx)
   
        # Loop over all batches
        for i in range(total_batch):
            
            batch_idx = total_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += wide_mlp_clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                       para_cur_lr)
        
#       learning rate decaying
#         para_cur_lr = para_cur_lr*0.98
        
        if epoch%para_eval_byepoch != 0:
            continue
    
        tmp_test_acc = wide_mlp_clf.inference(dxtest, cxtest, ytest, para_keep_prob) 
        tmp_train_acc = wide_mlp_clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
    
    print "Optimization Finished!"

loss on epoch  0  :  248.414669037 [0.072632946] [0.066301525]
loss on epoch  10  :  182.041134516 [0.20881972] [0.20668781]
loss on epoch  20  :  119.78852484 [0.21530479] [0.22629]
loss on epoch  30  :  72.4136905317 [0.21011673] [0.2369559]
loss on epoch  40  :  41.4497991491 [0.21919584] [0.24747767]
loss on epoch  50  :  23.5136919728 [0.22049287] [0.2587201]
loss on epoch  60  :  14.113192885 [0.21660182] [0.27371001]
loss on epoch  70  :  9.43611409929 [0.22178988] [0.28106081]
loss on epoch  80  :  6.98401696152 [0.22438392] [0.29403287]
loss on epoch  90  :  5.54885720765 [0.22049287] [0.30570769]
loss on epoch  100  :  4.6190436637 [0.230869] [0.31781495]
loss on epoch  110  :  3.96468808033 [0.24513619] [0.32804844]
loss on epoch  120  :  3.48226148884 [0.24902724] [0.33914673]
loss on epoch  130  :  3.11341652385 [0.25162128] [0.34736234]
loss on epoch  140  :  2.82550982303 [0.27237353] [0.35500145]
loss on epoch  150  :  2.59551349062 [0.27496758] [0.3672528]
loss on epoc

In [ ]:
# wide, embedding, co-occurrence and feature interaction 

class wide_embed_coInter_NN():
    
    def __init__(self, n_conti, n_disc, n_class,\
                 n_disc_voca, session,n_embedding, n_hidden_list, lr, l2):
        
        self.LEARNING_RATE = lr
                
        self.N_CLASS = n_class
        self.N_CONTI = n_conti
        self.N_DISC = n_disc
        self.N_DISC_VOCA = n_disc_voca
        self.N_EMBED = n_embedding
        self.L2 = l2
        
        self.N_HIDDEN_LAYERS = len(n_hidden_list)
        self.N_TOTAL = self.N_CONTI+ self.N_DISC*self.N_EMBED
        self.N_TOTAL_EMBED = self.N_DISC*self.N_EMBED

        self.sess=session
        
        self.dx = tf.placeholder(tf.int32, [None, self.N_DISC])
        self.cx = tf.placeholder(tf.float32, [None, self.N_CONTI])
        self.y = tf.placeholder(tf.float32, [None, self.N_CLASS])
        self.lr = tf.placeholder(tf.float32, shape=())
        self.keep_prob = tf.placeholder(tf.float32)
        
#       wide part on categorical features
        self.dx_trans = tf.transpose( self.dx, [1,0] )
        
        for i in range(self.N_DISC):
            
            with tf.variable_scope("wide"+str(i)):
                w= tf.Variable(\
                tf.random_uniform([self.N_DISC_VOCA[i], self.N_CLASS],-1.0, 1.0) )
#                         tf.random_normal([self.N_DISC_VOCA, self.N_CLASS],\
#                         stddev=math.sqrt(2.0/float(self.N_DISC_VOCA))) )
                if i==0:
                    dx_wsum = tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
#                   L2
                    self.regularizer_wide = tf.nn.l2_loss(w)
                    
                else:
                    dx_wsum = dx_wsum + tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
#                   L2  
                    self.regularizer_wide = self.regularizer_wide + tf.nn.l2_loss(w)
                    
            
#       embedding of categorical features        
        tmp_embeded = []                
        for i in range(self.N_DISC):
            
            with tf.variable_scope("embedding"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA[i], self.N_EMBED],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )
                              
                tmp_embeded.append( tf.nn.embedding_lookup( w, self.dx_trans[i] ) )
                
        dx_embeded = tf.concat(tmp_embeded, 1)
        x_concate =  tf.concat( [self.cx,dx_embeded], 1 )
        
#       co-occurrence of categorical features
        for i in range(20):
            
            tmpvoca1 = self.N_DISC_VOCA[i] 
            
            for j in range(i+1,20):
                
                tmpvoca2 = self.N_DISC_VOCA[j]
                
                with tf.variable_scope("cooccur"+str(i)+str(j)):
                    
                    
#                   vector approximate of co-occurrence matrix 
                    w1 = tf.Variable(tf.random_normal([self.N_CLASS, tmpvoca1, 1],\
                                            stddev=math.sqrt(2.0/float( tmpvoca1 ))) )
                    
                    w2 = tf.Variable(tf.random_normal([self.N_CLASS, 1, tmpvoca2],\
                                            stddev=math.sqrt(2.0/float( tmpvoca2 ))) )
                    
                    cooc_mat = []
                    for k in range(self.N_CLASS):
                        cooc_mat.append( tf.reshape( tf.matmul(w1[k],w2[k]), [-1,1] ) )
                        
#                   n_class by voca1*voca2  
                    cooc_vec = tf.stack(cooc_mat)
                    cooc_vec = tf.transpose(cooc_mat,[1,0])
                    
#                   build the idx of co-occurrence features
                    cooc_idx = self.dx_trans[i] * tmpvoca2 + self.dx_trans[j]
    
                    if i==0 and j==1:
                        cooc_wsum  = tf.nn.embedding_lookup( cooc_vec, cooc_idx )
                        self.regularizer_cooc   = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
                    else:
                        cooc_wsum += tf.nn.embedding_lookup( cooc_vec, cooc_idx )
                        self.regularizer_cooc += (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
        
        
#       feature interaction
        for i in range(self.N_DISC):
            
            with tf.variable_scope("interact"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA[i], self.N_CONTI],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )

                tmp_interact.append( tf.nn.embedding_lookup( w, self.dx_trans[i] ) )
    
                if i==0:
                    inter_wsum   = tf.nn.embedding_lookup( w, self.dx_trans[i] )
            
                else:
                    inter_wsum + = tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    

#       embedding hidden layers
        with tf.variable_scope("embed_h0"):
            w= tf.Variable(tf.random_normal([self.N_TOTAL_EMBED, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(self.N_CONTI)))) 
            b= tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            h_embed = tf.nn.relu( tf.add( tf.matmul(x_concate, w),b) )
                
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("embed_h"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                b= tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                h_embed = tf.nn.relu( tf.add( tf.matmul(h_embed, w),b) )
                
#       L2   
        self.regularizer = tf.nn.l2_loss(w)

        
#       interaction hidden layers
        inter_w = []
        for i in range( n_hidden_list[0] ):
            inter_w.append(inter_wsum)
        
        inter_w = tf.stack(inter_w)
        inter_w = tf.transpose( inter_w, [1,0])

        
        with tf.variable_scope("inter_h0"):
            w= tf.Variable(tf.random_normal([self.N_CONTI, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(self.N_CONTI)))) 
            
            w= w + inter_w
            
            b= tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            h_inter ten= tf.nn.relu( tf.add( tf.matmul(x_concate, w),b) )
                
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("inter_h"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                b= tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                h_inter = tf.nn.relu( tf.add( tf.matmul(h_inter, w),b) )
        
#       dropout
#       h = tf.nn.dropout(h, self.keep_prob)

#       L2
        self.regularizer + = tf.nn.l2_loss(w)
        
#       output layer  
        h = h_embed + h_inter
    
        with tf.variable_scope("output"):
            
            w= tf.Variable(tf.random_normal([n_hidden_list[self.N_HIDDEN_LAYERS-1],\
                                             self.N_CLASS],\
            stddev=math.sqrt(2.0/float(n_hidden_list[self.N_HIDDEN_LAYERS-1])))) 
            b= tf.Variable(tf.zeros( [ self.N_CLASS ] ))
            
            h = tf.add( tf.matmul(h, w), dx_wsum )
            h = tf.add( h, cooc_wsum )
            h = tf.add( h, b )


            self.logit = h
     
    
#           L2  
            self.regularizer + = tf.nn.l2_loss(w) 
    
#       overall L2
        self.regularizer + = (self.regularizer_wide + self.regularizer_cooc)
         
    
#   initialize loss and optimization operations for training
    def train_ini(self):
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                                    logits=self.logit, labels=self.y) \
                                  + self.L2*self.regularizer )
        self.optimizer = \
        tf.train.AdadeltaOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#       tf.train.RMSPropOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdadeltaOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.GradientDescentOptimizer(learning_rate = self.lr).minimize(self.cost)
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
#   training on batch of data
    def train_batch(self, dx_batch,cx_batch, y_batch, keep_prob, lr):
        
        _,c = sess.run([self.optimizer,self.cost],\
                       feed_dict={self.dx:dx_batch, \
                                  self.cx:cx_batch, self.y:y_batch,\
                                  self.lr:lr,\
                                  self.keep_prob:keep_prob\
                                 })
        
        return c
    
#   initialize inference         
    def inference_ini(self):
        self.correct_prediction = tf.equal(tf.argmax(self.logit,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, "float"))
        
#   infer givn testing data    
    def inference(self, dx_test, cx_test, y_test, keep_prob):
        return sess.run([self.accuracy], feed_dict={self.dx:dx_test,\
                                                  self.cx:cx_test, self.y:y_test,\
                                                  self.keep_prob:keep_prob\
                                                   })
        
#   unit_test
    def test(self, dx_test, cx_test, y_test ):
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        return sess.run( [self.tmpshape], \
                 feed_dict={self.dx:dx_test, self.cx:cx_test, self.y:y_test})

